In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
class MyDecisionTree:

    pass

In [6]:
X = np.array([
    [1, 2, 2],
    [2, 1, 0],
    [1, 0, 0]
])

In [18]:
def gini_impurity(X):

    m, n = X.shape

    for j in range(n):
        uniques = np.unique(X[:, j])
        for elem in uniques:
            dictacion = {}
            for i in range(m):
                if X[i, j] in dictacion:
                    dictacion[X[i, j]] += 1
                else:
                    dictacion[X[i, j]] = 1
            gini = 1
            for values in dictacion.values():
                gini -= (values / n) ** 2
            print(gini)


print(gini_impurity(X))

0.4444444444444445
0.4444444444444445
0.6666666666666665
0.6666666666666665
0.6666666666666665
0.4444444444444444
0.4444444444444444
None
